# Round 5 Counterparty Analysis

This notebook helps you analyze Round 5 counterparty trade data across multiple days.

**Steps included**:
1. Load and concatenate daily trade CSVs  
2. Adjust timestamps to a common scale  
3. Compute summary statistics per counterparty  
4. (Optional) Merge in mid-price data for P&L analysis  
5. Time-series and clustering examples

In [2]:
import pandas as pd

# File paths and corresponding days
files = [
    ('/Users/ynakhla/Desktop/imc/r5/data/trades_round_5_day_2.csv', 2),
    ('/Users/ynakhla/Desktop/imc/r5/data/trades_round_5_day_3.csv', 3),
    ('/Users/ynakhla/Desktop/imc/r5/data/trades_round_5_day_4.csv', 4),
]

# Load and preprocess each day's data
dfs = []
for filepath, day in files:
    df_day = pd.read_csv(filepath, sep=';')
    df_day['absolute_timestamp'] = df_day['timestamp'] + (day - 1) * 1_000_000
    df_day['tick'] = df_day['absolute_timestamp'] // 100
    df_day['day'] = day
    dfs.append(df_day)

# Concatenate into a single DataFrame
combined = pd.concat(dfs, ignore_index=True)

# Display basic info and sample
print("Combined shape:", combined.shape)
combined.head()

Combined shape: (53477, 10)


,timestamp,buyer,seller,symbol,currency,price,quantity,absolute_timestamp,tick,day
0,0,Caesar,Paris,CROISSANTS,SEASHELLS,4265.0,8,1000000,10000,2
1,100,Charlie,Paris,RAINFOREST_RESIN,SEASHELLS,9999.0,1,1000100,10001,2
2,200,Paris,Caesar,CROISSANTS,SEASHELLS,4266.0,8,1000200,10002,2
3,200,Paris,Caesar,JAMS,SEASHELLS,6520.0,7,1000200,10002,2
4,200,Paris,Charlie,KELP,SEASHELLS,2046.0,1,1000200,10002,2


In [3]:
# Summary statistics per counterparty (buyer + seller)
summary_buyer = combined.groupby('buyer').agg(
    trades_as_buyer=('quantity', 'count'),
    volume_as_buyer=('quantity', 'sum'),
    avg_price_as_buyer=('price', 'mean')
)

summary_seller = combined.groupby('seller').agg(
    trades_as_seller=('quantity', 'count'),
    volume_as_seller=('quantity', 'sum'),
    avg_price_as_seller=('price', 'mean')
)

summary = summary_buyer.join(summary_seller, how='outer').fillna(0)
summary['total_trades'] = summary['trades_as_buyer'] + summary['trades_as_seller']
summary['total_volume'] = summary['volume_as_buyer'] + summary['volume_as_seller']

# Display top 5 most active counterparties
summary.sort_values('total_trades', ascending=False).head()

,trades_as_buyer,volume_as_buyer,avg_price_as_buyer,trades_as_seller,volume_as_seller,avg_price_as_seller,total_trades,total_volume
Caesar,12598.0,55726.0,8385.026909,16931,132125,5165.898825,29529.0,187851.0
Paris,10512.0,29531.0,5303.649258,12135,31965,4819.798434,22647.0,61496.0
Camilla,13053.0,136275.0,3858.740979,5885,36863,7709.741716,18938.0,173138.0
Charlie,10123.0,33711.0,4455.900622,7183,26978,4856.132674,17306.0,60689.0
Pablo,2141.0,6750.0,16494.777674,5672,30151,11289.754584,7813.0,36901.0
